<h1> Titanics Dataset Predictions </h1>
<h2> Jose M. Serra Jr. </h2>

In [20]:
import matplotlib.pyplot as mplplt
import numpy as np
import pandas as pd
from scipy.stats import reciprocal
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.optimizers.schedules import *
import tensorflow.keras.callbacks as tkc
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.manifold import Isomap
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.utils import resample

In [21]:
file1_train, file2_test = pd.read_csv(r"D:\Datasets\Titanic\train.csv", delimiter=","),\
pd.read_csv(r"D:\Datasets\Titanic\test.csv", delimiter= ",")

<p> Now I am going to encode the Sex column into a binary 1s, and 0s output Sex_Binary column.</p>

In [24]:
file1_train = file1_train.drop(columns = ["PassengerId","Name"])
X, y = file1_train.drop("Survived", axis=1), file1_train["Survived"]

In [25]:
file1_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,211536,13.0000,NaN,S
887,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,1,male,26.0,0,0,111369,30.0000,C148,C


In [26]:
objs = X.select_dtypes(["object"])
num  = X.select_dtypes(["number"])

In [27]:
objs.isnull().sum().head()

Sex           0
Ticket        0
Cabin       687
Embarked      2
dtype: int64

In [28]:
num.isnull().sum().head()

Pclass      0
Age       177
SibSp       0
Parch       0
Fare        0
dtype: int64

In [29]:
numerical_features = num.columns
numerical_pipeline = Pipeline(
    steps=[
            ("imputer", SimpleImputer(strategy = 'mean')),
            ("scaler", StandardScaler())
])

In [30]:
for i in range(len(objs.columns)):
    print(objs.columns[i], objs.iloc[:,i].value_counts().unique())

Sex [577 314]
Ticket [7 6 5 4 3 2 1]
Cabin [4 3 2 1]
Embarked [644 168  77]


In [31]:
binary = ["Sex"]
binary_pipeline = Pipeline(steps=[("binary", OneHotEncoder())])

In [32]:
cat1 = ["Ticket", "Cabin", "Embarked"]
catergorical_pipeline = Pipeline(steps=[("imputer", SimpleImputer(strategy = 'most_frequent')), ("ordinal_encoder", OrdinalEncoder()),("scaler", StandardScaler())])

In [33]:
data_preprocessor = ColumnTransformer( [('numerical', numerical_pipeline, numerical_features),
                                      ('binary', binary_pipeline, binary),
                                      ('categorical', catergorical_pipeline, cat1)])

In [34]:
X

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,female,35.0,1,0,113803,53.1000,C123,S
4,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,211536,13.0000,NaN,S
887,1,female,19.0,0,0,112053,30.0000,B42,S
888,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,male,26.0,0,0,111369,30.0000,C148,C


In [35]:
X = Pipeline(steps=[('processing',data_preprocessor)]).fit_transform(X)

In [36]:
param_distribs = {'learn_rate' : np.array(np.linspace(.1,.9))}

In [70]:
def base_model1(learn_rate = .1):
    input_dim = X.shape[1]
    model =Sequential([
    Dense(200 , input_dim = input_dim, activation= "relu"),
    Dropout(rate=.10),
    Dense(100, activation= "tanh"),
    Dense(1,activation = "sigmoid"),
    ])
    lr_schedule = ExponentialDecay(
    learn_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)
    model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics=["accuracy"])
    return model

In [71]:
checkpoint = [ModelCheckpoint("Titanic.h5", monitor='accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')]
early = EarlyStopping(monitor='accuracy', min_delta=0, patience=10, verbose=1, mode='auto')

In [72]:
NN_clf = KerasClassifier(build_fn=base_model1, epochs=100, verbose=1, callbacks =[checkpoint,early] )#callbacks=[lr_scheduler])
#NN_clf._estimator_type = "classifier"

In [76]:
random_trainor = GridSearchCV(estimator=NN_clf,param_grid=param_distribs, cv=None)
random_trainor.fit((X,Reconstructions),(y.values.reshape(-1,1)))

ValueError: Found input variables with inconsistent numbers of samples: [2, 891]

In [46]:
best = random_trainor.best_estimator_.model

In [47]:
best.summary()

Model: "sequential_256"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_768 (Dense)            (None, 200)               2200      
_________________________________________________________________
dropout_256 (Dropout)        (None, 200)               0         
_________________________________________________________________
dense_769 (Dense)            (None, 100)               20100     
_________________________________________________________________
dense_770 (Dense)            (None, 1)                 101       
Total params: 22,401
Trainable params: 22,401
Non-trainable params: 0
_________________________________________________________________


In [49]:
best.save("Titanic.h5")

In [48]:
X_test = data_preprocessor.fit_transform(file2_test)
X_test

array([[ 0.87348191,  0.3349926 , -0.49947002, ..., -0.26949361,
        -0.35149045, -0.47091535],
       [ 0.87348191,  1.32553003,  0.61699237, ...,  0.37293425,
        -0.35149045,  0.70076689],
       [-0.31581919,  2.51417495, -0.49947002, ..., -1.00502694,
        -0.35149045, -0.47091535],
       ...,
       [ 0.87348191,  0.65196458, -0.49947002, ...,  1.53675282,
        -0.35149045,  0.70076689],
       [ 0.87348191,  0.        , -0.49947002, ...,  0.3636237 ,
        -0.35149045,  0.70076689],
       [ 0.87348191,  0.        ,  0.61699237, ..., -0.70708939,
        -0.35149045, -1.64259759]])

In [50]:
PassengerId = file2_test["PassengerId"].to_list()

In [51]:
best.predict(X_test)

array([[0.04392037],
       [0.2218095 ],
       [0.19832149],
       [0.02812907],
       [0.6128014 ],
       [0.03252152],
       [0.37740767],
       [0.09421116],
       [0.2721239 ],
       [0.03116572],
       [0.02812907],
       [0.01799607],
       [0.97670496],
       [0.09445503],
       [0.95531166],
       [0.7196944 ],
       [0.06423452],
       [0.09824595],
       [0.62595403],
       [0.24903092],
       [0.01989615],
       [0.21785879],
       [0.99645275],
       [0.00474545],
       [0.9450077 ],
       [0.02029982],
       [0.80060625],
       [0.09824595],
       [0.19832149],
       [0.09824595],
       [0.01235852],
       [0.03584003],
       [0.6833513 ],
       [0.6833513 ],
       [0.19832149],
       [0.09824595],
       [0.37740767],
       [0.37740767],
       [0.02812907],
       [0.19832149],
       [0.28868622],
       [0.16847512],
       [0.00676227],
       [0.3113969 ],
       [0.9865422 ],
       [0.03252152],
       [0.01989615],
       [0.031

In [52]:
final_pred = (best.predict(X_test) > 0.5).astype("int32").flatten()

In [53]:
final_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [54]:
d = {"PassengerId":PassengerId, "Survived":final_pred}
pd.DataFrame(data=d,index=None, columns= ["PassengerId","Survived"]).to_csv("12232021.csv",index=False, header=1)